# Computação Bioinspirada 2024-02

Este repositório contém o trabalho da Tópicos Avançados em Ciências de Computação II
(Computação Bioinspirada, no semestre 2024-02).

O trabalho consiste em trabalhar com um conjunto de dados multirrótulo onde cada instância representa uma sequência de
proteína. Cada rótulo (classe) corresponde a uma localização subcelular e as proteínas podem estar presentes
simultaneamente em dois ou mais compartimentos celulares. O conjunto de dados possui seis localizações
subcelulares: Proteínas do Capsídeo Viral, Proteínas da Membrana Celular do Hospedeiro, Proteínas do Retículo
Endoplasmático do Hospedeiro, Proteínas do Citoplasma do Hospedeiro, Proteínas do Núcleo do Hospedeiro e
Proteínas Secretadas. As colunas representam os códigos de Gene Ontology (relacionados à função da proteína),
com valores que indicam a frequência do código para cada proteína. As seis últimas colunas indicam a presença
(1) ou ausência (0) da proteína em cada uma das localizações subcelulares mencionadas.

Desenvolvemos um modelo de classificação multirrótulo usando **redes neurais artificiais**.

Existem dois conjuntos de dados: um de vírus e um de plantas.


### Imports 

In [148]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [149]:
random_seed = 42
tf.random.set_seed(random_seed)
tf.keras.utils.set_random_seed(random_seed)

### Lendo os dados dos datasets e separando em treino e teste

In [150]:
plants_dataset = pd.read_csv('./Plants_Dataset_Term_Frequency.tsv', sep='\t', skiprows=1).iloc[:, 1:]
virus_dataset =  pd.read_csv('./Virus_Dataset_Term_Frequency.tsv', sep='\t', skiprows=1).iloc[:, 1:]

In [151]:
# Separar as features e os rótulos para o dataset de plantas
X_plants = plants_dataset.iloc[:, :-6].values  # Todas as colunas, exceto as últimas 6 e a primeira
y_plants = plants_dataset.iloc[:, -6:].values  # As últimas 6 colunas (rótulos)

# Separar as features e os rótulos para o dataset de vírus
X_virus = virus_dataset.iloc[:, :-6].values
y_virus = virus_dataset.iloc[:, -6:].values

# Dividir dados em treino e teste
X_train_plants, X_test_plants, y_train_plants, y_test_plants = train_test_split(
    X_plants, y_plants, test_size=0.2, random_state=random_seed
)

X_train_virus, X_test_virus, y_train_virus, y_test_virus = train_test_split(
    X_virus, y_virus, test_size=0.2, random_state=random_seed
)

In [152]:

# Padronizar os dados
scaler_plants = StandardScaler()
X_train_plants = scaler_plants.fit_transform(X_train_plants)
X_test_plants = scaler_plants.transform(X_test_plants)

scaler_virus = StandardScaler()
X_train_virus = scaler_virus.fit_transform(X_train_virus)
X_test_virus = scaler_virus.transform(X_test_virus)

In [153]:
# Arquitetura da rede neural
def create_model(input_shape, output_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(32, activation='relu'),
        Dense(output_shape, activation='sigmoid')  # Saída com sigmoid para classificação binária
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [154]:
# Criar e treinar o modelo para plantas
model_plants = create_model(X_train_plants.shape[1], y_train_plants.shape[1])
history_plants = model_plants.fit(X_train_plants, y_train_plants, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
20/20 [==============================] - 1s 9ms/step - loss: 0.6921 - accuracy: 0.1831 - val_loss: 0.5993 - val_accuracy: 0.2968
Epoch 2/50
20/20 [==============================] - 0s 4ms/step - loss: 0.2966 - accuracy: 0.2934 - val_loss: 0.4175 - val_accuracy: 0.3097
Epoch 3/50
20/20 [==============================] - 0s 4ms/step - loss: 0.1636 - accuracy: 0.3614 - val_loss: 0.3382 - val_accuracy: 0.2581
Epoch 4/50
20/20 [==============================] - 0s 4ms/step - loss: 0.1105 - accuracy: 0.4230 - val_loss: 0.3063 - val_accuracy: 0.2774
Epoch 5/50
20/20 [==============================] - 0s 4ms/step - loss: 0.0810 - accuracy: 0.4279 - val_loss: 0.2816 - val_accuracy: 0.2774
Epoch 6/50
20/20 [==============================] - 0s 4ms/step - loss: 0.0622 - accuracy: 0.4522 - val_loss: 0.2657 - val_accuracy: 0.2903
Epoch 7/50
20/20 [==============================] - 0s 4ms/step - loss: 0.0486 - accuracy: 0.4603 - val_loss: 0.2511 - val_accuracy: 0.3097
Epoch 8/50
20/20 [==

In [155]:
# Criar e treinar o modelo para vírus
model_virus = create_model(X_train_virus.shape[1], y_train_virus.shape[1])
history_virus = model_virus.fit(X_train_virus, y_train_virus, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
5/5 [==============================] - 1s 28ms/step - loss: 0.8241 - accuracy: 0.1603 - val_loss: 0.7287 - val_accuracy: 0.3030
Epoch 2/50
5/5 [==============================] - 0s 7ms/step - loss: 0.6750 - accuracy: 0.3053 - val_loss: 0.6721 - val_accuracy: 0.3030
Epoch 3/50
5/5 [==============================] - 0s 7ms/step - loss: 0.5777 - accuracy: 0.3969 - val_loss: 0.6260 - val_accuracy: 0.3030
Epoch 4/50
5/5 [==============================] - 0s 7ms/step - loss: 0.5073 - accuracy: 0.4962 - val_loss: 0.5872 - val_accuracy: 0.3333
Epoch 5/50
5/5 [==============================] - 0s 7ms/step - loss: 0.4516 - accuracy: 0.5573 - val_loss: 0.5520 - val_accuracy: 0.3939
Epoch 6/50
5/5 [==============================] - 0s 7ms/step - loss: 0.4055 - accuracy: 0.5954 - val_loss: 0.5219 - val_accuracy: 0.3939
Epoch 7/50
5/5 [==============================] - 0s 7ms/step - loss: 0.3663 - accuracy: 0.6565 - val_loss: 0.4992 - val_accuracy: 0.4242
Epoch 8/50
5/5 [=================

In [156]:
# Obter previsões para o conjunto de teste de plantas
predictions_plants_test = model_plants.predict(X_test_plants)

# Obter previsões para o conjunto de treino de plantas
predictions_plants_train = model_plants.predict(X_train_plants)


# Obter previsões para o conjunto de teste de vírus
predictions_virus_test = model_virus.predict(X_test_virus)

# Obter previsões para o conjunto de treino de vírus
predictions_virus_train = model_virus.predict(X_train_virus)


# Transformar as probabilidades em rótulos binários (0 ou 1) com um limiar de 0.5
predicted_classes_plants_test = (predictions_plants_test > 0.5).astype(int)
predicted_classes_plants_train = (predictions_plants_train > 0.5).astype(int)
predicted_classes_virus_test = (predictions_virus_test > 0.5).astype(int)
predicted_classes_virus_train = (predictions_virus_train > 0.5).astype(int)


6/6 [==============================] - 0s 1ms/step


In [157]:
def hamming_loss(predictions, actual_values):
    N = len(predictions)
    L = 6

    s = 0

    for i in range(N):
        for j in range(L):
            s += predictions[i][j] ^ actual_values[i][j]
    
    return s / (N * L)

In [158]:
possibilidades = [
    [predicted_classes_plants_train, y_train_plants, 'dados de treino das plantas'],
    [predicted_classes_plants_test, y_test_plants, 'dados de teste das plantas'],
    [predicted_classes_virus_train, y_train_virus, 'dados de treino dos vírus'],
    [predicted_classes_virus_test, y_test_virus, 'dados de teste dos vírus']
]

In [159]:
for possibilidade in possibilidades:
    print(f'A hamming loss nos {possibilidade[-1]} foi de {100*hamming_loss(*possibilidade[:-1]):.2f}%')

A hamming loss nos dados de treino das plantas foi de 1.38%
A hamming loss nos dados de teste das plantas foi de 2.85%
A hamming loss nos dados de treino dos vírus foi de 3.05%
A hamming loss nos dados de teste dos vírus foi de 6.10%
